# Experimentación:

Link del colab para reproducir las composiciones: https://colab.research.google.com/drive/16XL1f98hLPu5q8xYzFARdDC6ErMYCwVY?authuser=1#scrollTo=08SXXu4-agmh

Vamos a poner a prueba la generación de música mediante la cadena de Markov (de varios órdenes) de diferentes maneras:
- Ver que en obras de Piano ambas manos tengan cierta cohesión.
- Dada una obra, que pueda hacer una adaptación que suene "agradable".
- Dadas varias obras de un cierto estilo musical (en este caso vals), ver que conserve las características de este.

Aclaramos que de cierto modo es subjetiva la manera de medir resultados; para un músico profesional probablemente no tenga ningún sentido la armonía, fraseo, dinámica o demás características de las obras generadas y es por esto que lo vemos más como espectadores sin un amplio conocimiento musical asombrados por los compases que se generan de manera pseudo artificial.

## Subir los archivos obtenidos en `tests`


In [18]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')


Saving generada_orden1.PNG to generada_orden1.PNG
User uploaded file "generada_orden1.PNG" with length 85759 bytes


## Librería para reproducir archivos midi

In [22]:
!pip install music21


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


### Ejemplo 1: Composición dada un vals a una cadena de orden 1

Vamos a generar una composición construyendo la matriz de transición únicamente a partir de una obra, el Vals en E menor de Griboyedov. Por algún motivo este archivo midi obtenido en MusesCore solo permite escuchar la voz de la mano derecha.

In [23]:
from music21 import converter, midi

def escuchar_music21(ruta_midi):
    mf = converter.parse(ruta_midi)
    return mf.show('midi')

escuchar_music21('waltz-in-e-minor-griboyedov.mid')

A continuación, reproducimos la composición generada con la cadena de Markov de orden 1, es decir, solo tiene en consideración al generar una nota la inmediatamente anterior a esta. En este caso sí se pueden escuchar ambas voces.

In [24]:
escuchar_music21('griboyedov_waltz_remix.mid')

Una vez escuchada la obra, podemos ver que se cumple el primer objetivo de los experimentos, pues se escucha con cierta cohesión entre ambas manos y que genera estructuras parecidas. Esto lo podemos analizar más en detalle si examinamos los primeros compases de ambas partituras. 

Generada:

![Partitura generada](generada_orden1.PNG)  

Original:

![Partitura generada](original.PNG) 



Si vemos más detalladamente, ciertos compases comparten cierta estructura aunque con pequeñas variaciones que son acordes a la armonía y la tonalidad de la obra. Destacamos unos en particular para poderlo apreciar mejor:

Generada:

![Partitura generada](generada_edit1.PNG)  

Original:

![Partitura generada](original_edit1.PNG) 

Otro aspecto que nos llamó la atención es que cambia la estructura de la mano izquierda vals: generalmente se toca una nota en el bajo y posteriormente se repite dos veces un acorde algunas octavas más arriba; en este caso toca primero las dos octavas y luego sí un bajo. Esto se puede deber al orden de la cadena, pero confirmémoslo haciendo una composición de orden 3.

### Ejemplo 2: Composición dada un vals a una cadena de orden 3

In [25]:
escuchar_music21('griboyedov_waltz_remix_order_3.mid')

![Partitura generada](generada_orden3.PNG) 

De aca escuchamos y vemos en la partitura que la primera parte de la composición la adapta mejor a la estructura del Vals, como se ve en las primeras notas señaladas en rojo. El segundo círculo rojo nos llama la atención ya que en la obra original en esa parte pasa a clave de Sol, pero en este caso decide seguir en clave de Fa. 

También decidimos experimentar con una de orden 2 y dio resultados similares puesto que si detecta los dos acordes seguidos procede a tocar el bajo en casi todas las ocasiones y si tenía el bajo y un acorde, sabía que debía tocar el otro acorde.

Con estos dos ejemplos vistos, podemos decir que logramos los dos primeros objetivos satisfactoriamente.

### Ejemplo 3: Composición de un vals en Sol Mayor.

Para este experimento buscamos en MusesCore diferentes archivos MXL de Valses en una tonalidad en específico, para que no hubiese cambios bruscos a la hora de la generación. Como hallar estos archivos es difícil (o costoso, pues en la mayoría de sitios tocaba pagar una subscripción), vamos a crear la matriz de transición a partir de 3 de estas obras:

Vals de Schubert:


In [26]:
escuchar_music21('waltz-in-g-major-d-844-franz-schubert.mid')

![Partitura generada](vals_schubert.PNG) 

Vals de Titov:

In [34]:
escuchar_music21('waltz-in-g-major-nikolai-titov-nikolaj-titov.mid')

![Partitura generada](vals_titov.PNG) 

Vals de Schumann:

In [28]:
escuchar_music21('waltz-for-piano-in-g-major-from-robert-schumanns-kinderball-op-130-no-2-but-arrangedtwo-hands.mid')

![Partitura generada](vals_schumann.PNG) 

El resultado para una cadena de orden 1 fue el siguiente:

In [29]:
escuchar_music21('3waltz_g_major_1765297402.mid')

![Partitura generada](3waltz_anotaded3.PNG) 

Para analizar esta pieza, nos parecieron curiosos dos detalles principalmente: 

El primero, las notas azules destacadas en la mano derecha son la célula rítmica del Vals, solo que se cambiaron las figuras, ya que el bajo son dos negras ligadas, seguidas de dos acordes, una blanca y de nuevo dos negras ligadas. Esto es interesante puesto que de los tres valses que se escogieron, el que tiene el ritmo más marcado es el de Titov, que está en 3/8, es decir en cada compás hay tres tiempos de corchea. Como la composición está generada en 4/4 = 8/8, interpretamos que la cadena "escaló" estos tres tiempos de corchea y los adaptó a pasar a tener 4/4 por compás. Además, las notas señaladas en verde parecen una combinación entre los valses de Schumann y Schubert puesto que combinan acordes de blanca seguidos de negra

El segundo, señalado en amarillo es que adopta la voz de la mano derecha del vals de Titov, pero de nuevo escalado para que coincidan las figuras con los tiempos del compás 4/4.

Veamos otra generación con estas mismas condiciones, pero subiendo el tempo a 180 para que suene más movido.

In [30]:
escuchar_music21('3waltz_g_major_1765299940.mid')

![Partitura generada](3waltz_2ndatt.PNG) 

Acá sí podemos apreciar mejor cómo las tres obras intervienen en la generación. Los compases señalados en rojo tienen claramente la estructura del vals de Schumann, el único que tiene tales acordes con alteraciones y una mano izquierda con varias negras y blancas por compás. La parte en azul se asemeja más al de Schubert, pues la mano izquierda no tiene tan marcado el ritmo del vals como el de Titov y la mano derecha va intercalando notas dobles con sencillas. Por último la sección verde sí tiene marcado el ritmo mencionado anteriormente y se pueden ver las secuencias de negras terminadas con una blanca, las frases de la voz principal del vals de Titov.

Concluimos de esto que así se haya hecho la misma simulación con las mismas obras y la misma matriz de transición, se pueden apreciar momentos diferentes de las obras incluidas. El principal motivo por el que conjeturamos esto es debido al vector de inicialización y su importancia en la parte inicial de la obra. Esto es así ya que intentamos probar con otro tipo de valses, pero sonaba predominantemente una sobre las otras (principalmente la que tuviese más notas), precisamente por este motivo. Para ejemplificar anexamos una generación de unos valses en Mi menor, pero no seleccionados tan correctamente como los previos.

In [31]:
escuchar_music21('3waltz_1765294849.mid')

In [32]:
escuchar_music21('3waltz_1765295671.mid')

Se puede ver cómo predomina el Vals en Mi menor de Chopin, opacando los otros con la que se construyó la matriz de transición.